In [2]:
# Dependencies
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
from pymongo import MongoClient
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced


In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client.stock_predictions
collection = db.stock_ticker
document = collection.find_one()
ticker = document['ticker']
ticker

'CVE'

In [9]:
data = yf.download(ticker, '2021-01-01', dt.datetime.date(dt.datetime.now()).isoformat(), interval='1d')
data.drop(['Adj Close', 'Volume'], axis=1, inplace=True)    
data['Day Result'] = np.where(data['Close'] > data['Open'], 1, 0)
df = pd.DataFrame(data)
df.drop(columns='Close', inplace=True)

[*********************100%***********************]  1 of 1 completed


In [7]:
# Encode the data
le = LabelEncoder()
df['Open'] = le.fit_transform(df['Open'])
df['High'] = le.fit_transform(df['High'])
df['Low'] = le.fit_transform(df['Low'])


In [8]:
# Split data into training and testing sets
X = df.drop(['Day Result'], axis=1)
y = df['Day Result'].values

In [10]:
# Build the logistic regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
# Scale and normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [12]:
# Naive Random Oversampling
ros = RandomOverSampler(random_state=0)
X_res, y_res = ros.fit_resample(X_train_scaled, y_train)

In [13]:
# Train the logistic regression model using resampled data
logreg = LogisticRegression(solver='lbfgs', random_state=1)
logreg.fit(X_res, y_res)
y_pred = logreg.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

/Users/ryanandrade/miniforge3/envs/DataScience/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.6162790697674418

In [14]:
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
predictions

,Actual,Predicted
0,0,0
1,1,1
2,1,1
3,0,1
4,1,1
...,...,...
82,0,0
83,1,1
84,1,1
85,0,1
